In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Books.csv")
df

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [18]:
df.duplicated().sum()

0

In [4]:
users = pd.read_csv("Users.csv")
ratings = pd.read_csv("Ratings.csv")

In [5]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [6]:
users.duplicated().sum()

0

##popularity based system

In [7]:
ratings_with_name = ratings.merge(df,on='ISBN')

In [8]:
num_rating_df=ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating' : 'num_rating'},inplace = True)
num_rating_df

,Book-Title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [9]:
ratings['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

In [ ]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].agg(lambda x: x.astype(float).mean()).reset_index()
avg_rating_df.rename(columns = {'Book-Rating' : 'avg_rating'}, inplace = True)
avg_rating_df

In [11]:
popular_df = num_rating_df.merge(avg_rating_df,on = 'Book-Title')

In [12]:
popular_df

,Book-Title,num_rating,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [32]:
popular_df = popular_df[popular_df['num_rating']>=250].sort_values('avg_rating',ascending = False).head(50)

In [37]:
 popular_df = popular_df.merge(df,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_rating','avg_rating']]

In [39]:
popular_df.shape

(50, 5)

##collaborative fitering based recommender system

In [40]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating']>200
padhe_likhe_users = x[x].index

In [44]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [48]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>200
famous_books = y[y].index

In [49]:
famous_books

Index(['A Painted House', 'A Time to Kill', 'Bridget Jones's Diary',
       'Divine Secrets of the Ya-Ya Sisterhood: A Novel',
       'Message in a Bottle', 'The Da Vinci Code', 'The Firm',
       'The Horse Whisperer', 'The Lovely Bones: A Novel',
       'The Nanny Diaries: A Novel', 'The Notebook', 'The Pelican Brief',
       'The Secret Life of Bees', 'Wild Animus'],
      dtype='object', name='Book-Title')

In [53]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]